In [1]:
!pip install google-generativeai
!pip install python-dotenv

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import threading
import time
import concurrent

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
load_dotenv()
api_key_1 = os.getenv('GOOGLE_API_KEY_1')
api_key_2 = os.getenv('GOOGLE_API_KEY_2')
api_key_3 = os.getenv('GOOGLE_API_KEY_3')
api_key_4 = os.getenv('GOOGLE_API_KEY_4')
api_key_5 = os.getenv('GOOGLE_API_KEY_5')
api_key_6 = os.getenv('GOOGLE_API_KEY_6')

api_keys = [
    api_key_1,
    api_key_2,
    api_key_3,
    api_key_4,
    api_key_5,
    api_key_6
]

In [4]:
filenames = [
    "group_1.txt",
    "group_2.txt",
    "group_3.txt",
    "group_4.txt",
    "group_5.txt",
    "group_6.txt",
    "group_7.txt",
    "group_8.txt",
    "group_9.txt",
    "group_10.txt",
    "group_11.txt",
    "group_12.txt",
]

In [5]:
file_contents = []

In [6]:
for filename in filenames:
    file = open(f"./data/{filename}", "r")
    file_content = file.read()
    file.close()
    file_contents.append(file_content)

In [7]:
question = "Kas ma võin autoga sõites alkoholi tarbida?"

In [8]:
system_prompt = "Sa oled seaduste abiline. Kui sulle antud seaduses on küsimusele vastust, siis vasta sellele. Kui vastus puudub, siis vasta '0'"

In [9]:
response_format = """kastuta vastamisel järgmist formaati:
<seaduse nimi>

<sinu vastus>"""

In [10]:
responses = []

In [16]:
#group_index = 0
#group_count = len(api_keys) * 2

#for api_key in api_keys:
#    genai.configure(api_key=api_key)
#
#    print(f"progress: {group_index + 1}/{group_count}")
#    file_content = file_contents[group_index]
#    model = genai.GenerativeModel("gemini-1.5-flash")
#    response_text = model.generate_content([
#        system_prompt,
#        response_format,
#        file_content,
#        "Küsimus on järgmine:",
#        question
#    ]).text
#    responses.append(response_text)

#    group_index += 1

#    print(f"progress: {group_index + 1}/{group_count}")
#    file_content = file_contents[group_index]
#    model = genai.GenerativeModel("gemini-1.5-flash-8b")
#    response_text = model.generate_content([
#        system_prompt,
#        response_format,
#        file_content,
#        "Küsimus on järgmine:",
#        question
#    ]).text
#    responses.append(response_text)

#    group_index += 1

In [45]:
thread_local = threading.local()
responses_lock = threading.Lock()
api_keys_lock = threading.Lock()
responses = []


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

def get_response(data):
    api_keys, file_content, group_index, group_count, system_prompt, response_format, question = data

    print(f"Starting: {group_index + 1}/{group_count}")
    
    with api_keys_lock:
        if not api_keys:
            print("No API keys left to process.")
            return
        api_key = api_keys.pop(0)
        
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel("gemini-1.5-flash")
    if group_index % 2 == 0:
        model = genai.GenerativeModel("gemini-1.5-flash-8b")

    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text

    # Safely append to the shared responses list
    with responses_lock:
        responses.append((group_index, response_text))
        
    print(f"Ending: {group_index + 1}/{group_count}")

        
def get_all_responses(api_keys, file_contents, system_prompt, response_format, question):
    group_count = len(api_keys)
    print(group_count)
    tasks = [
        ( api_keys, file_contents[i], i, group_count, system_prompt, response_format, question)
        for i in range(group_count)
    ]

    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        executor.map(get_response, tasks)

if __name__ == "__main__":

    temp_api_keys = api_keys.copy()
    temp_api_keys = temp_api_keys * 2
    temp_api_keys.sort()

    start_time = time.time()

    get_all_responses(temp_api_keys, file_contents, system_prompt, response_format, question)
    
    duration = time.time() - start_time

    responses.sort(key=lambda x: x[0])
    for idx, response_text in responses:
        print(f"Response {idx + 1}: {response_text}")

    print(f"I did it in {duration} seconds")


12
Starting: 1/12
Starting: 2/12
Starting: 3/12
Starting: 4/12
Starting: 5/12
Starting: 6/12
Starting: 7/12
Ending: 4/12
Starting: 8/12
Ending: 2/12
Starting: 9/12
Ending: 8/12
Starting: 10/12
Ending: 10/12
Starting: 11/12
Ending: 3/12
Starting: 12/12
Ending: 1/12
Ending: 5/12
Ending: 12/12
Ending: 9/12
Ending: 7/12
Ending: 11/12
Response 1: <Karistusseadustik>

0

Response 2: Liiklusseadus

Ei, autoga sõites ei tohi alkoholi tarbida.  Käesoleva seaduse § 69 sätestab, et juht ei tohi olla alkoholi piirmäära ületavas seisundis ega joobeseisundis.

Response 3: <Karistusseadustik>

0

Response 4: Meresõiduohutuse seadus

Ei, Meresõiduohutuse seaduse § 45 lõike 4 ja § 37 lõike 2 alusel on väikelaeva või muu veesõiduki juhtimisel keelatud olla joobeseisundis.  Alkoholi tarvitamine autoga sõitmisel ei ole otseselt reguleeritud, aga kuna sõidukijuht peab olema alati võimeline sõidukit ohutult juhtima, siis ei ole alkoholi tarvitamine sõidu ajal soovitatav.

Response 5: <Karistusseadustiku §-d

In [46]:
for response in responses:
    print(response)

(0, '<Karistusseadustik>\n\n0\n')
(1, 'Liiklusseadus\n\nEi, autoga sõites ei tohi alkoholi tarbida.  Käesoleva seaduse § 69 sätestab, et juht ei tohi olla alkoholi piirmäära ületavas seisundis ega joobeseisundis.\n')
(2, '<Karistusseadustik>\n\n0\n')
(3, 'Meresõiduohutuse seadus\n\nEi, Meresõiduohutuse seaduse § 45 lõike 4 ja § 37 lõike 2 alusel on väikelaeva või muu veesõiduki juhtimisel keelatud olla joobeseisundis.  Alkoholi tarvitamine autoga sõitmisel ei ole otseselt reguleeritud, aga kuna sõidukijuht peab olema alati võimeline sõidukit ohutult juhtima, siis ei ole alkoholi tarvitamine sõidu ajal soovitatav.\n')
(4, '<Karistusseadustiku §-d>\n\n0\n')
(6, '<Korrakaitseseadus>\n\n0\n')
(7, 'Liikluskindlustuse seadus\n\nEi, liikluskindlustuse seaduse § 8 lõike 2 punkti 4 alusel ei ole lubatud juhtida sõidukit joobeseisundis.\n')
(8, 'Alkoholiseadus\n\n0\n')
(9, 'Liiklusseadus\n\n0\n')
(10, '<Alkoholi-, tubaka-, kütuse- ja elektriaktsiisi seadus>\n\n0\n')
(11, '0\n')


In [47]:
genai.configure(api_key=api_keys[0])
model = genai.GenerativeModel("gemini-1.5-pro")
response_texts = [element[1] for element in responses]

prompt = (
    ["Sulle antakse mitme erineva mudeli vastused erinevatest seadustest, tsiteeri mulle mudeli vastuseid, mis pole '0'"] +
    [response_format] +
    response_texts + 
    ["Küsimus on järgnev:"] +
    [question]
)

best_answer = model.generate_content(prompt)

In [48]:
print(best_answer.text)

Liiklusseadus

Ei, autoga sõites ei tohi alkoholi tarbida.  Käesoleva seaduse § 69 sätestab, et juht ei tohi olla alkoholi piirmäära ületavas seisundis ega joobeseisundis.
Meresõiduohutuse seadus

Ei, Meresõiduohutuse seaduse § 45 lõike 4 ja § 37 lõike 2 alusel on väikelaeva või muu veesõiduki juhtimisel keelatud olla joobeseisundis.  Alkoholi tarvitamine autoga sõitmisel ei ole otseselt reguleeritud, aga kuna sõidukijuht peab olema alati võimeline sõidukit ohutult juhtima, siis ei ole alkoholi tarvitamine sõidu ajal soovitatav.
Liikluskindlustuse seadus

Ei, liikluskindlustuse seaduse § 8 lõike 2 punkti 4 alusel ei ole lubatud juhtida sõidukit joobeseisundis.



---
# Testing chat ui
---

In [ ]:
!pip install ipywidgets

In [49]:
%%html

<style>
body {
  font-family: Arial, sans-serif;
  margin: 0;
  padding: 0;
  display: flex;
  justify-content: center;
  align-items: center;
  height: 100vh;
  background-color: #f4f4f9;
}

.chat-container {
  width: 90%;
  max-width: 400px;
  height: 500px;
  background: #fff;
  box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
  border-radius: 10px;
  display: flex;
  flex-direction: column;
}

.chat-box {
  flex-grow: 1;
  padding: 10px;
  overflow-y: auto;
  border-bottom: 1px solid #ddd;
}

.chat-box::-webkit-scrollbar {
  width: 6px;
}

.chat-box::-webkit-scrollbar-thumb {
  background: #ddd;
  border-radius: 3px;
}

.message {
  margin: 5px 0;
  padding: 10px;
  border-radius: 8px;
  max-width: 75%;
  word-wrap: break-word;
}

.user-message {
  background-color: #d1e7dd;
  align-self: flex-end;
}

.bot-message {
  background-color: #f8d7da;
  align-self: flex-start;
}

.chat-input {
  display: flex;
  padding: 10px;
  border-top: 1px solid #ddd;
  background: #f9f9f9;
}

.chat-input input {
  flex-grow: 1;
  padding: 10px;
  border: 1px solid #ccc;
  border-radius: 5px;
  font-size: 16px;
  margin-right: 10px;
}

.chat-input button {
  padding: 10px 15px;
  background: #007bff;
  color: white;
  border: none;
  border-radius: 5px;
  cursor: pointer;
  font-size: 16px;
}

.chat-input button:hover {
  background: #0056b3;
}
</style>

In [58]:
from IPython.display import Javascript
import json
from time import sleep

def respond_to_prompt(prompt):
    # Simulate processing delay
    sleep(2)
    return f"Response to: {prompt}"

# Function to handle incoming JavaScript calls
def handle_prompt(prompt):
    response = respond_to_prompt(prompt)  # Call the Python function
    # Send the response back to JavaScript
    display(Javascript(f"removeLoading(); addMessage({json.dumps(response)}, false);"))


In [59]:
%%html

<html>
<body>
<div class="chat-container">
    <div id="chat-box" class="chat-box"></div>
    <div class="chat-input">
        <input id="chat-input" type="text" placeholder="Type a message..." />
        <button id="send-button">Send</button>
    </div>
</div>
<script>
    const chatBox = document.getElementById('chat-box');
    const chatInput = document.getElementById('chat-input');
    const sendButton = document.getElementById('send-button');

    // Function to add messages to the chat
    function addMessage(content, isUser = true) {
        const message = document.createElement('div');
        message.classList.add('message', isUser ? 'user-message' : 'bot-message');
        message.textContent = content;
        chatBox.appendChild(message);
        chatBox.scrollTop = chatBox.scrollHeight; // Scroll to bottom
    }

    // Add loading icon
    function addLoading() {
        const loading = document.createElement('div');
        loading.classList.add('loading');
        loading.textContent = '...';
        loading.id = 'loading';
        chatBox.appendChild(loading);
        chatBox.scrollTop = chatBox.scrollHeight; // Scroll to bottom
    }

    // Remove loading icon
    function removeLoading() {
        const loading = document.getElementById('loading');
        if (loading) loading.remove();
    }

    // Handle send button click
    sendButton.addEventListener('click', () => {
        const userMessage = chatInput.value.trim();
        if (userMessage) {
            addMessage(userMessage, true); // Add user message
            chatInput.value = ''; // Clear input
            addLoading(); // Add loading icon

            // Call the Python function via IPython kernel
            const pythonCode = `handle_prompt(${JSON.stringify(userMessage)})`;
            IPython.notebook.kernel.execute(pythonCode);
        }
    });
</script>
</body>
</html>

In [61]:
import ipywidgets as widgets
from IPython.display import display

# Define the Python function
def my_function(event=None):
    print("Button clicked! Running my_function...")

# Create a button widget
button = widgets.Button(
    description="Click Me",
    button_style="",  # You can use 'success', 'info', 'warning', or 'danger' for styles
    tooltip="Click this button to run a function",
    icon="check",  # Add an icon from FontAwesome, e.g., 'check', 'plus', etc.
)

# Link the button to the function
button.on_click(my_function)

# Display the button in the notebook
display(button)

Button(description='Click Me', icon='check', style=ButtonStyle(), tooltip='Click this button to run a function…

In [20]:
%%html
<html>
<head>
  <link rel="stylesheet" href="styles.css">
</head>
<body>
<div class="header">
    <h1>juur<span>.ai</span></h1>
</div>

<div class="input-footer">
    <textarea type="text" placeholder="Ask anything" class="input-text glassmorphism"> </textarea>
</div>
</body>
</html>

In [62]:
pip install notebookjs

Note: you may need to restart the kernel to use updated packages.
